# CALCULATE TRAVEL TIME EVERY TRIP
## BUS/PT TRAVELTIME INCLUDE PERSON IN BUS/PT AND PERSON WALK, WAIT TO PT/BUS

In [1]:
import yaml
import pandas as pd
import numpy as np
from lxml import etree
import csv

import os

# Set up event and vehicle types

### After scenario

In [2]:
# TRANSIT_VEHICLE_PATH = "data\\after\\output_transitVehicles.xml"
# EVENT_PATH = "data\\after\\output_events.xml"

# OUTPUT_EVENT_AFTER_PROCESSOR = "data\\after\\scoring\\travel_time\\veh_traveltime.csv"

### Before Scenario

In [3]:
TRANSIT_VEHICLE_PATH = "data\\before\\output_transitVehicles.xml"
EVENT_PATH = "data\\before\\output_events.xml"

OUTPUT_EVENT_AFTER_PROCESSOR = "data\\before\\scoring\\travel_time\\veh_traveltime.csv"

In [4]:
# TRANSIT_VEHICLE_PATH = "data\\simple_scenario\\transitVehicles.xml"
# EVENT_PATH = "data\\simple_scenario\\output\\output_events.xml"

# OUTPUT_EVENT_AFTER_PROCESSOR = "data\\simple_scenario\\scoring\\traveltime\\veh_traveltime.csv"


# TRANSIT_VEHICLE_PATH = "data\\real\\transitVehicles.xml"
# EVENT_PATH = "data\\real\\output_events_Hai.xml"

# OUTPUT_EVENT_AFTER_PROCESSOR = "data\\real\\scoring\\travel_time\\veh_traveltime.csv"



In [5]:

if os.path.exists(OUTPUT_EVENT_AFTER_PROCESSOR):
    os.remove(OUTPUT_EVENT_AFTER_PROCESSOR)

folder_path = os.path.dirname(OUTPUT_EVENT_AFTER_PROCESSOR)
os.makedirs(folder_path, exist_ok=True)

# Xử lý file transit vehicle

In [6]:
!powershell -Command "Get-Content 'data\\simple_scenario\\transitVehicles.xml' -TotalCount 20 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

5: <?xml version="1.0" encoding="UTF-8"?>
5: <vehicleDefinitions xmlns="http://www.matsim.org/files/dtd" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.matsim.org/files/dtd http://www.matsim.org/files/dtd/vehicleDefinitions_v1.0.xsd">
5:   <vehicleType id="bus_type"> <capacity><seats persons="50"/><standingRoom persons="0"/></capacity> <length meter="10.0"/> </vehicleType>
5:   <vehicleType id="tram_type"> <capacity><seats persons="50"/><standingRoom persons="0"/></capacity> <length meter="10.0"/> </vehicleType>
5:   <vehicleType id="train_type"> <capacity><seats persons="100"/><standingRoom persons="0"/></capacity> <length meter="10.0"/> </vehicleType>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_0" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_1" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_2" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_3" type="bus_type"/>
5:  <vehicle id="veh_Bus_Line_1_dir_fwd_4" type="bus_typ

### Xử lý ra dict chứa id và type

In [7]:
tree = etree.parse(TRANSIT_VEHICLE_PATH)
root = tree.getroot()

#### xlmn trong thẻ vehicleDefinitions  là namespace cho tất cả các tag trong tag này. nghĩa là tên đầy đủ của tag là {http://www.matsim.org/files/dtd}vehicleDefinitions, {http://www.matsim.org/files/dtd}vehicle.

#### Khi dùng lxml phải dùng namespace: root.xpath("//m:vehicleDefinitions/m:vehicle", namespaces=ns)

In [8]:
ns = {'m': 'http://www.matsim.org/files/dtd'}
vehtype_dict = {}


for node in root.xpath("//m:vehicleDefinitions/m:vehicle", namespaces=ns):
    id = node.xpath("@id")[0]
    type = node.xpath("@type")[0]
    print([id,type])
    vehtype_dict[id] = type



['1_0', 'tram_93pax']
['1_1', 'tram_93pax']
['1_2', 'tram_93pax']
['1_3', 'tram_93pax']
['1_4', 'tram_93pax']
['2_0', 'tram_93pax']
['2_1', 'tram_93pax']
['2_2', 'tram_93pax']
['2_3', 'tram_93pax']
['2_4', 'tram_93pax']
['3_0', 'tram_93pax']
['3_1', 'tram_93pax']
['3_2', 'tram_93pax']
['3_3', 'tram_93pax']
['3_4', 'tram_93pax']
['4_0', 'tram_93pax']
['4_1', 'tram_93pax']
['4_2', 'tram_93pax']
['4_3', 'tram_93pax']
['4_4', 'tram_93pax']
['rb43_0', 'train']
['rb43_1', 'train']
['rb43_2', 'train']
['rb43_3', 'train']
['rb43_4', 'train']
['re11_0', 'train']
['re11_1', 'train']
['re11_2', 'train']
['re11_3', 'train']
['re11_4', 'train']
['re2_0', 'train']
['re2_1', 'train']
['re2_2', 'train']
['re2_3', 'train']
['re2_4', 'train']
['veh_bus_10_in_07:00:00', 'bus_90pax']
['veh_bus_10_in_07:15:00', 'bus_90pax']
['veh_bus_10_in_07:30:00', 'bus_90pax']
['veh_bus_10_in_07:45:00', 'bus_90pax']
['veh_bus_10_in_08:00:00', 'bus_90pax']
['veh_bus_10_in_08:15:00', 'bus_90pax']
['veh_bus_10_in_08:30:00'

## Xử lý event

In [9]:
!powershell -Command "Get-Content 'data\\simple_scenario\\output\\output_events.xml' -TotalCount 1000 | ForEach-Object { '{0:5}: {1}' -f $_.ReadCount, $_ }"

5: <?xml version="1.0" encoding="utf-8"?>
5: <events version="1.0">
5: 	<event time="21600.0" type="TransitDriverStarts" driverId="pt_veh_Tram_Line_1_dir_fwd_0_tram_type" vehicleId="veh_Tram_Line_1_dir_fwd_0" transitLineId="Tram_Line_1" transitRouteId="dir_fwd" departureId="veh_Tram_Line_1_dir_fwd_0"  />
5: 	<event time="21600.0" type="departure" person="pt_veh_Tram_Line_1_dir_fwd_0_tram_type" link="7" legMode="car"  />
5: 	<event time="21600.0" type="PersonEntersVehicle" person="pt_veh_Tram_Line_1_dir_fwd_0_tram_type" vehicle="veh_Tram_Line_1_dir_fwd_0"  />
5: 	<event time="21600.0" type="TransitDriverStarts" driverId="pt_veh_Tram_Line_1_dir_bwd_0_tram_type" vehicleId="veh_Tram_Line_1_dir_bwd_0" transitLineId="Tram_Line_1" transitRouteId="dir_bwd" departureId="veh_Tram_Line_1_dir_bwd_0"  />
5: 	<event time="21600.0" type="departure" person="pt_veh_Tram_Line_1_dir_bwd_0_tram_type" link="74" legMode="car"  />
5: 	<event time="21600.0" type="PersonEntersVehicle" person="pt_veh_Tram_Line_

#### Chỉ đếm những người lên xe bus khác lái xe

In [10]:
hint_bus_type = "bus"
schema = ['vehIdList', 'vehicleTypeList', 'mainMode' , 'startTime', 'travelTime']
person_trip_map = {}

with open(OUTPUT_EVENT_AFTER_PROCESSOR, 'a') as f:
    for x in schema[:-1]:
        f.write(x + ",")
    f.write(schema[-1] + "\n")

context = etree.iterparse(EVENT_PATH, events=('end',))
for event, elem in context:
    if elem.tag == "event":
        e_type = elem.get("type")

        if e_type == "departure":
            personId = elem.get("person")

            # Nếu trip của người này đã được ghi nhận trước đó và chưa kết thúc thì bỏ qua
            if personId in person_trip_map.keys():
                continue
            # Nếu là người lái xe bus thì bỏ qua
            if personId.startswith("pt_"):
                continue

            veh_id = elem.get("vehicle")
            time = elem.get("time")
            mainMode = elem.get("computationalRoutingMode")

            person_trip_map[personId] = {
                "vehIdList": [],
                "vehicleTypeList": [],
                "mainMode": mainMode,
                "startTime": float(time),
                "travelTime": 0
                }


        elif e_type == "PersonEntersVehicle":
            personId = elem.get("person")
            veh_id = elem.get("vehicle")

            if personId not in person_trip_map.keys():
                continue
            if personId.startswith("pt_"):
                continue
                
            person_trip_map[personId]["vehIdList"].append(veh_id)

            if veh_id in vehtype_dict.keys():
                person_trip_map[personId]["vehicleTypeList"].append(vehtype_dict[veh_id])
            else :
                person_trip_map[personId]["vehicleTypeList"].append("undefined")


        elif e_type == "actstart":
            personId = elem.get("person")
            veh_id = elem.get("vehicle")

            if personId not in person_trip_map.keys():
                continue
            if personId.startswith("pt_"):
                continue
            if elem.get("actType") == "pt interaction":
                continue
        
            time = elem.get("time")
            travel_time = float(time) - person_trip_map[personId]["startTime"]
            person_trip_map[personId]["travelTime"] = travel_time

            # Nếu người này chỉ đi bộ nhưng vẫn có computationalRoutingMode là pt/car thì bỏ qua
            if len(person_trip_map[personId]["vehIdList"]) == 0 or len(person_trip_map[personId]["vehicleTypeList"]) == 0:
                del person_trip_map[personId]
        
            else:
                with open(OUTPUT_EVENT_AFTER_PROCESSOR, 'a', newline='') as f:
                    writer = csv.writer(f)
                    
                    # Chuẩn bị dữ liệu
                    # Biến list thành chuỗi, ví dụ dùng dấu ';' để phân cách các ID bên trong
                    veh_ids = ";".join(map(str, person_trip_map[personId]['vehIdList']))
                    veh_types = ";".join(map(str, person_trip_map[personId]['vehicleTypeList']))
                    
                    row = [
                        veh_ids,
                        veh_types,
                        person_trip_map[personId]['mainMode'],
                        person_trip_map[personId]['startTime'],
                        person_trip_map[personId]['travelTime']
                    ]
                    
                    # Ghi cả dòng một lần duy nhất
                    writer.writerow(row)
                del person_trip_map[personId]

    elem.clear()

print("Finished processing travel data.")





Finished processing travel data.


# Calculate bus, car avg travel time per trip
### if "bus" string in VehTypeList, then consider this trip is a bus trip ( even this trip also has tram, train, car )

In [11]:
def calculate_average_travel_time(person_trip_path: str):
    df = pd.read_csv(person_trip_path)
    busdf = df[(df['vehicleTypeList'].str.contains('bus'))]

    average_bus_travel_time = busdf['travelTime'].mean()
    bus_trip = len(busdf)
    average_car_travel_time = df[(df['mainMode'] == 'car')]['travelTime'].mean()
    car_trip = len(df[(df['mainMode'] == 'car')])
    
    return average_bus_travel_time, average_car_travel_time, bus_trip, car_trip

average_bus_time, average_car_time, bus_trip, car_trip = calculate_average_travel_time(OUTPUT_EVENT_AFTER_PROCESSOR)
print(f"Average bus travel time: {average_bus_time} seconds, in {bus_trip} trips")
print(f"Average car travel time: {average_car_time} seconds, in {car_trip} trips")

Average bus travel time: 3248.1112769132383 seconds, in 4691 trips
Average car travel time: 1193.8830615721586 seconds, in 16436 trips
